# Complete Simulation Chain for Visibility Calculation

## Read in Config

In [1]:
import vipy.simulation.utils as ut
import vipy.layouts.layouts as layouts
import astropy.constants as const
from astropy import units as un
import time as t

In [2]:
rc = ut.read_config('/net/nfshome/home/sfroese/vipy/config/default.toml')
array_layout = layouts.get_array_layout('eht')

src_crd = rc['src_coord']
wave1 = const.c/((float(rc['channel'].split(':')[0])-float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter
wave2 = const.c/((float(rc['channel'].split(':')[0])+float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter

## Generate (l,m)-plane / FOV

In [3]:
import vipy.simulation.scan as scan

In [4]:
grid = scan.create_bgrid(rc['fov_size'], 256, src_crd)
lm = scan.lm(grid, src_crd)

## Calculate Start and Stop times for every measurement

In [5]:
time = ut.calc_time_steps(rc)
time.shape

(2161,)

## Calculate Baselines for one measurement

In [6]:
baselines = scan.get_baselines(src_crd, time[:4], array_layout)

## Calculate Mueller/Jones matrices

In [7]:
import numpy as np
import torch

In [8]:
%%time
start = t.time()
torch.set_num_threads(8)
JJ_f1 = scan.getJones(lm, baselines, wave1, time, src_crd, array_layout)
JJ_f2 = scan.getJones(lm, baselines, wave2, time, src_crd, array_layout)

/scratch/sfroese/envs/radionets/lib/python3.8/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/net/nfshome/home/sfroese/vipy/vipy/simulation/scan.py:295: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  K = torch.tensor(getK(baselines,lm,wave))


CPU times: user 16.9 s, sys: 8.49 s, total: 25.4 s
Wall time: 3.51 s


## Open Source Image and compute Stokes vector

In [9]:
from astropy.io import fits
import matplotlib.pyplot as plt
import torch

In [10]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))
# plt.imshow((I[...,1]+I[...,2]).real)
# plt.colorbar()

## Integration

In [11]:
%%time
delta_t = rc['corr_int_time']
delta_f = float(rc['channel'].split(':')[1])*10**6
delta_l = lm[255,0,0]-lm[0,0,0]
delta_m = lm[0,255,0]-lm[0,0,0]
print(I.dtype)

integral = scan.integrate(JJ_f1, JJ_f2, I, 28, delta_t, delta_f, delta_l, delta_m)
integral.shape
end = t.time()
print(end - start)

torch.complex128
4.721632480621338
CPU times: user 4.54 s, sys: 3.1 s, total: 7.64 s
Wall time: 956 ms


In [12]:
integral

tensor([[ 5.6631e-04+5.7756e-06j, -6.9986e-04-7.1378e-06j,
          1.7107e-03+1.7447e-05j, -2.1141e-03-2.1561e-05j],
        [-7.0403e-05-1.9676e-04j, -1.4164e-04-3.9586e-04j,
         -2.9162e-05-8.1501e-05j, -5.8670e-05-1.6397e-04j],
        [ 1.0252e-03-6.5015e-04j,  1.1273e-04-7.1486e-05j,
         -9.5298e-04+6.0433e-04j, -1.0478e-04+6.6449e-05j],
        [-1.8037e-06+1.4401e-04j,  1.6767e-06-1.3386e-04j,
         -1.0021e-05+8.0003e-04j,  9.3144e-06-7.4366e-04j],
        [-1.1349e-03+5.1392e-04j, -2.2833e-03+1.0339e-03j,
         -1.2479e-04+5.6507e-05j, -2.5105e-04+1.1369e-04j],
        [ 8.2093e-04+1.7786e-03j,  9.0264e-05+1.9557e-04j,
          4.5606e-03+9.8810e-03j,  5.0146e-04+1.0865e-03j],
        [-1.1285e-04-2.9881e-04j, -1.2408e-05-3.2855e-05j,
         -4.6744e-05-1.2377e-04j, -5.1396e-06-1.3609e-05j],
        [ 2.6601e-04+1.8006e-04j,  5.3518e-04+3.6225e-04j,
         -2.4726e-04-1.6737e-04j, -4.9747e-04-3.3672e-04j],
        [-9.6173e-04-1.3505e-04j,  1.1885e-03+1.